<a href="https://colab.research.google.com/github/peterbabulik/QuantumWalker/blob/main/Algorithmic_Folding_QW_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.3 MB/s eta 0:00:00


In [2]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector, partial_trace, entropy
import matplotlib.pyplot as plt

# --- Simulation Parameters ---
NUM_FOLDS = 4 # Number of "Planck times" or folding steps after the initial seed

# --- Helper to visualize state ---
def visualize_state(qc, title=""):
    sv = Statevector(qc)
    print(f"\n--- {title} ---")
    print(sv.draw(output='latex_source')) # For rich display if available
    # For text display:
    # for i, amp in enumerate(sv.data):
    #     if not np.isclose(amp, 0):
    #         print(f"{format(i, f'0{qc.num_qubits}b')}: {amp:.3f}")
    probs = sv.probabilities_dict()
    print("Probabilities:", {k: f"{v:.3f}" for k, v in probs.items() if v > 1e-5})


# --- t_P1: Initial Seed ---
print("=== Planck Time 1 (t_P1): Initial Seed ===")
q_seed = QuantumRegister(1, 'q0')
qc_t1 = QuantumCircuit(q_seed) # Starts in |0>
# qc_t1.x(q_seed[0]) # Could start in |1>
# qc_t1.h(q_seed[0]) # Could start in |+>
visualize_state(qc_t1, "State at t_P1 (1 qubit)")
current_num_qubits = 1
current_registers = [q_seed]
current_circuit = qc_t1

# --- Algorithmic Folds ---
all_circuits_history = [qc_t1.copy()]

for fold_step in range(1, NUM_FOLDS + 1):
    print(f"\n=== Planck Time {fold_step + 1} (t_P{fold_step + 1}): Fold #{fold_step} ===")

    # For this simple model, let's say each existing qubit "pairs" with a new one
    # and creates a Bell pair with it. This doubles the qubits at each step.

    new_num_qubits = current_num_qubits * 2
    new_registers = []

    # Define new quantum registers for all qubits in this step
    # We need to be careful with how registers are named and managed if we build one big circuit
    # For simplicity, let's create a new circuit at each fold based on the previous state conceptually

    # Let's model "folding rule": each qubit from previous step interacts with a new ancilla
    # and forms a Bell pair. If previous state was |psi_prev>, new is |psi_prev> (tensor) U_entangle_all

    # More practically for simulation step-by-step:
    # We are building one circuit that grows

    if fold_step == 1: # First fold: q0 interacts with a new q1
        q_new = QuantumRegister(1, f'q{current_num_qubits}')
        new_registers = current_registers + [q_new]

        # Create a new circuit with the combined registers
        qc_fold = QuantumCircuit(*new_registers) # Unpack list of registers

        # Copy operations from previous circuit (if any beyond initialization)
        # For t_P1 -> t_P2, current_circuit is just the |0> state on q0
        # If current_circuit was complex, we'd append it:
        # qc_fold.append(current_circuit.to_instruction(), [r[i] for r in current_registers for i in range(r.size)])

        # Apply H to the "original" qubit (q0)
        qc_fold.h(new_registers[0][0]) # q0 is new_registers[0]
        # Apply CX between original (q0) and new (q1)
        qc_fold.cx(new_registers[0][0], new_registers[1][0]) # q1 is new_registers[1]

        current_circuit = qc_fold
        current_num_qubits = new_num_qubits
        current_registers = new_registers

    elif fold_step > 1:
        # General fold: each existing qubit q_i pairs with a new qubit q_{N+i}
        # This assumes we double the number of qubits at each step for this model

        prev_qubits_for_mapping = []
        for reg in current_registers:
            for i in range(reg.size):
                prev_qubits_for_mapping.append(reg[i])

        new_ancilla_registers = [QuantumRegister(1, f'q{current_num_qubits + i}') for i in range(current_num_qubits)]
        all_current_qregs_for_new_qc = current_registers + new_ancilla_registers

        qc_fold = QuantumCircuit(*all_current_qregs_for_new_qc)

        # Append the state preparation from the previous fold
        qc_fold.append(current_circuit.to_instruction(), prev_qubits_for_mapping)

        # Now entangle each "old" qubit with its "new" partner
        for i in range(current_num_qubits):
            original_qubit = prev_qubits_for_mapping[i] # This is a Qubit object
            new_ancilla_qubit = new_ancilla_registers[i][0] # This is a Qubit object

            qc_fold.h(original_qubit)
            qc_fold.cx(original_qubit, new_ancilla_qubit)

        current_circuit = qc_fold
        current_registers = all_current_qregs_for_new_qc
        current_num_qubits *= 2 # Number of qubits doubles

    all_circuits_history.append(current_circuit.copy())
    visualize_state(current_circuit, f"State at t_P{fold_step + 1} ({current_num_qubits} qubits)")

    # --- Calculate Entanglement ---
    # For simplicity, let's calculate bipartite entanglement between the "original block"
    # and the "newly added block" of qubits if fold_step >= 1
    if current_num_qubits >= 2:
        sv_current = Statevector(current_circuit)

        # Example: Entanglement between first half and second half of qubits
        # This gets tricky if qubits are not added in a simple tensor product way
        # Let's consider the entanglement of the first qubit with the rest as a simple measure

        if current_num_qubits > 1:
            # Trace out all qubits except the first one
            qubits_to_trace_out = list(range(1, current_num_qubits))
            if qubits_to_trace_out: # Ensure there's something to trace
                try:
                    rho_first_qubit = partial_trace(sv_current, qubits_to_trace_out)
                    ent_first_qubit = entropy(rho_first_qubit, base=2)
                    print(f"Entanglement of q0 with rest: {ent_first_qubit:.4f} bits")
                except Exception as e_ent:
                    print(f"Could not calculate entanglement: {e_ent}")


# --- Plot final circuit (if not too large) ---
final_qc_to_draw = all_circuits_history[-1]
if final_qc_to_draw.num_qubits <= 5: # Only draw if manageable
    print("\n--- Final Circuit ---")
    try:
        display(final_qc_to_draw.draw(output='mpl', fold=-1)) # Use display for Jupyter
    except:
        print(final_qc_to_draw.draw(output='text', fold=-1))
else:
    print(f"\nFinal circuit has {final_qc_to_draw.num_qubits} qubits, too large to draw neatly.")

=== Planck Time 1 (t_P1): Initial Seed ===

--- State at t_P1 (1 qubit) ---
 |0\rangle
Probabilities: {np.str_('0'): '1.000'}

=== Planck Time 2 (t_P2): Fold #1 ===

--- State at t_P2 (2 qubits) ---
\frac{\sqrt{2}}{2} |00\rangle+\frac{\sqrt{2}}{2} |11\rangle
Probabilities: {np.str_('00'): '0.500', np.str_('11'): '0.500'}
Entanglement of q0 with rest: 1.0000 bits

=== Planck Time 3 (t_P3): Fold #2 ===

--- State at t_P3 (4 qubits) ---
\frac{\sqrt{2}}{2} |0000\rangle+\frac{\sqrt{2}}{2} |1111\rangle
Probabilities: {np.str_('0000'): '0.500', np.str_('1111'): '0.500'}
Entanglement of q0 with rest: 1.0000 bits

=== Planck Time 4 (t_P4): Fold #3 ===

--- State at t_P4 (8 qubits) ---
\frac{\sqrt{2}}{4} |00000000\rangle+\frac{\sqrt{2}}{4} |00110011\rangle+\frac{\sqrt{2}}{4} |01010101\rangle+\frac{\sqrt{2}}{4} |01100110\rangle+\frac{\sqrt{2}}{4} |10011001\rangle+\frac{\sqrt{2}}{4} |10101010\rangle+\frac{\sqrt{2}}{4} |11001100\rangle+\frac{\sqrt{2}}{4} |11111111\rangle
Probabilities: {np.str_('00

Short Description: "A Qiskit toy model illustrating the 'algorithmic folding' tenet of an information-based cosmology. Shows how a minimal quantum information state can evolve into complex, highly entangled multi-qubit states through simple iterative rules, simulating conceptual early 'Planck time' steps."